In [1]:
import numpy as np
import numpy.ma as ma
import os
import pandas as pd
import astropy.units as u
from astropy import coordinates, cosmology, units
from photutils.aperture import EllipticalAperture, SkyCircularAperture, CircularAperture, aperture_photometry, ApertureStats
from astropy.io import ascii, fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip
import subprocess
import matplotlib.pyplot as plt
from astropath import path
from astropath import localization
from astropath import chance
from astropath import bayesian
from astropy.stats import SigmaClip
from photutils.background import Background2D, MedianBackground

# standard imports for my work
from bpt_utils import *
from read_transients_data import *
from correct_redshift_evolution import *
from generate_bkg_galaxies import *
from helper_functions import *

import seaborn as sns
plt.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
    }
)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['image.origin'] = 'lower'

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

sns.set_context('talk') 
sns.set(font_scale=1.4)
sns.set_palette('colorblind')
sns.set_style('ticks')

plt.rcParams["font.family"] = "Times New Roman"
%matplotlib inline

In [2]:
def get_rad(i):
    """ RA, DEC in degrees.
    E_RA, E_DEC in degrees.
    """

    Path = path.PATH()
    if not tabr.iloc[i]['RADEC']:
        return None
    co_rad = coordinates.SkyCoord(tabr.iloc[i]['RADEC'])
    eellipse = dict(a=tabr.iloc[i]['E_RA'], b=tabr.iloc[i]['E_DEC'], theta=0.)
    Path.init_localization('eellipse', center_coord=co_rad, eellipse=eellipse)
    return Path, co_rad


def set_priors(Path, tabo, Pcand='inverse', PU=0., Poffset='exp', Poffsetrad=6):
    if tabo is None:
        return None
      
    Path.init_candidates(tabo['raMean'].tolist(),
                         tabo['decMean'].tolist(),
                         tabo['rKronRad'].tolist(),
                         mag=tabo['rKronMag'].tolist())
    Path.init_cand_prior(Pcand, P_U=PU)
    Path.init_theta_prior(Poffset, Poffsetrad, scale)
    Path.calc_priors()
    return None


def run_prob(Path):
    P_Ox, P_Ux = Path.calc_posteriors('fixed', box_hwidth=30., max_radius=30)



In [3]:
# This notebook is specifically for legacy host associations
nickname = 'mizu'
# Note - Koyaanisqatsi second possible host is not listed in legacy galaxy catalog, had to be added in manually
# Note - Erdos second possible host is listed as a star in legacy galaxy catalog, had to be manually modified
imageName = 'frbs_data/dsa_results/{}/r.fits'.format(nickname)
position0, e_ra, e_dec = get_ra_dec_errs(nickname)
tabr = pd.DataFrame(zip([position0], [e_ra], [e_dec]), columns=['RADEC', 'E_RA', 'E_DEC'])
Path, co_rad = get_rad(0)

In [4]:
def load_fits_file(file_path):
    with fits.open(file_path, memmap=True) as hdu_list:
        data = hdu_list[1].data
        # Check endianness
        if data.dtype.byteorder not in ('=', '|'):
            data = data.byteswap().newbyteorder()
        tabo = Table(data)
    return tabo

tabo = load_fits_file("frbs_data/dsa_results/{}/{}_cat.fits".format(nickname, nickname))
sel = (tabo["type"] == "REX") + (tabo["type"] == "DEV")+ (tabo["type"] == "EXP") + (tabo["type"] == "SER")
tabo1 = tabo[sel]
tabo1["mag_r"] = np.log10(tabo1["flux_r"]*1e-9)/-0.4

co_gals = coordinates.SkyCoord(ra=tabo1["ra"], dec=tabo1["dec"], unit="deg")
sep = np.array(co_gals.separation(co_rad).to_value("arcsec"), dtype="<f4")
raMean = np.array(tabo1["ra"], dtype="<f4")
decMean = np.array(tabo1["dec"], dtype="<f4")
rKronMag = np.array(tabo1["mag_r"], dtype="<f4")
rKronRad = np.array(tabo1["shape_r"], dtype="<f4")

tabo = pd.DataFrame({"raMean": raMean, "decMean": decMean, "rKronMag": rKronMag, 
                     "rKronRad": rKronRad, "sep": sep}).reset_index()
tabo = tabo[tabo["sep"]<30] # Pick all galaxies within 30 arcsec
tabo


,index,raMean,decMean,rKronMag,rKronRad,sep
0,0,143.968613,73.286591,23.553823,0.667175,17.326143
1,1,143.973541,73.286072,22.268047,0.474292,11.914756
2,2,143.979355,73.287193,23.196890,0.844649,10.270014
3,3,143.984589,73.284950,15.440931,7.429181,1.220825
4,4,143.989578,73.283813,22.209066,0.352957,6.572427


In [5]:
# path params
scale = 1
PU0 = 0.1
Pcand = 'inverse'
Poffset = 'exp'
Poffsetrad = 6

In [6]:
if tabo is not None:
    set_priors(Path, tabo, PU=PU0, Pcand=Pcand, Poffset=Poffset, Poffsetrad=Poffsetrad)
    run_prob(Path)
    df = Path.candidates
else:
    print("No associations found")
    df = pd.DataFrame()
df = df.sort_values("P_Ox", ascending=False).reset_index()    
df.head()

,index,ra,dec,ang_size,mag,P_O,p_xO,P_Ox,P_Ux
0,3,143.984589,73.284950,7.429181,15.440931,0.898527,2.343388e-03,9.869794e-01,0.013021
1,4,143.989578,73.283813,0.352957,22.209066,0.000548,2.250619e-08,5.778634e-09,0.013021
2,2,143.979355,73.287193,0.844649,23.196890,0.000233,3.684725e-10,4.016537e-11,0.013021
3,1,143.973541,73.286072,0.474292,22.268047,0.000520,2.572351e-22,6.265704e-23,0.013021
4,0,143.968613,73.286591,0.667175,23.553823,0.000173,4.947876e-43,4.010757e-44,0.013021
